В этой задаче мы прогнозируем продажи товаров в различных магазинах в течение двух 28-дневных периодов времени.

Files
* calendar.csv - содержит информацию о датах продажи товаров.
* sales_train_validation.csv - содержит исторические ежедневные данные о единичных продажах для каждого продукта и магазина [d_1 - d_1913]
* sample_submission.csv - правильный формат для отправки ответов.
* sell_prices.csv - содержит информацию о цене проданных товаров в каждом магазине и дате их продажи.
* sales_train_evaluation.csv - доступно за месяц до окончания конкурса. Будет включать продажи [d_1 - d_1941]

File 1: “calendar.csv” содержит информацию о датах продажи товаров. 
* date: дата в формате “y-m-d”. 
* wm_yr_wk: идентификатор недели, к которой относится дата. 
* weekday: тип дня (суббота, воскресенье,..., пятница). 
* wday: идентификатор рабочего дня, начиная с субботы. 
* month: месяц даты. 
* year: год даты. 
* event_name_1: если дата включает в себя событие, то имя этого события. 
* event_type_1: если дата включает в себя событие, то тип этого события. 
* event_name_2: если дата включает в себя второе событие, то имя этого события. 
* event_type_2: если дата включает в себя второе событие, то тип этого события. 
* snap_CA, snap_TX, and snap_WI: двоичная переменная (0 или 1), указывающая, разрешают ли магазины CA, TX или WI совершать покупки SNAP на исследуемую дату. 1 указывает, что разрешены срочные покупки.

File 2: “sell_prices.csv” Содержит информацию о цене продаваемых товаров в каждом магазине и дате их продажи. 
* store_id: идентификатор магазина, в котором продается товар. 
* item_id: идентификатор продукта. 
* wm_yr_wk: идентификатор недели. 
* sell_price: цена товара за данную неделю / магазин. Цена указана за неделю (в среднем за семь дней). Если он отсутствует, это означает, что продукт не был продан в течение исследуемой недели. Обратите внимание, что хотя цены являются постоянными на еженедельной основе, они могут меняться с течением времени (как обучение, так и набор тестов).

File 3: “sales_train.csv” содержит исторические ежедневные данные о единичных продажах для каждого продукта и магазина. 
* item_id: идентификатор продукта. 
* dept_id: идентификатор отдела, к которому принадлежит продукт. 
* cat_id: идентификатор категории, к которой относится продукт. 
* store_id: идентификатор магазина, в котором продается товар. 
* state_id: состояние, в котором находится магазин. 
* d_1, d_2, …, d_i, … d_1941: количество единиц, проданных в день i, Начиная с 2011-01-29.

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [ ]:
## Seeder
# seed to make all processes deterministic
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

**Помощник для загрузки данных по идентификатору хранилища**

In [ ]:
# Чтение данных
# store - идентификатор магазина
def get_data_by_store(store):
    # Чтение c использованием функции concat
    df = pd.concat(
        [
            pd.read_pickle(BASE),
            pd.read_pickle(PRICE).iloc[:,2:],
            pd.read_pickle(CALENDAR).iloc[:,2:]
        ],
        axis=1
    )
    
    # Оставляем только соответствующий магазин
    df = df[df['store_id'] == store]

    # Из-за ограничения памяти мы должны читать лаги и средние характеристики отдельно и отбрасывать элементы, которые нам не нужны.
    # Поскольку наши сетки объектов выровнены, мы можем использовать индекс, чтобы сохранить только необходимые строки.
    # Выравнивание хорошо для нас, так как concat использует меньше памяти, чем merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # чтобы не достичь предела памяти
    
    df = pd.concat([df, df3], axis=1)
    del df3 # чтобы не достичь предела памяти
    
    # Создание списка особенностей
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET] + features]
    
    # Пропуск первых n строк
    df = df[df['d'] >= START_TRAIN].reset_index(drop=True)
    
    return df, features

In [ ]:
# Рекомбинированный тестовый набор после обучения
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_' + store_id + '.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test

**Создание динамических лагов**

In [ ]:
def make_lag(LAG_DAY):
    lag_df = base_test[['id', 'd', TARGET]]
    col_name = 'sales_lag_' + str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [ ]:
def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id', 'd', TARGET]]
    col_name = 'rolling_mean_tmp_' + str(shift_day) + '_' + str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

**Параметры модели**

In [ ]:
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'tweedie',
    'tweedie_variance_power': 1.1,
    'metric': 'rmse',
    'subsample': 0.5,
    'subsample_freq': 1,
    'learning_rate': 0.03,
    'num_leaves': 2**11-1,
    'min_data_in_leaf': 2**12-1,
    'feature_fraction': 0.5,
    'max_bin': 100,
    'n_estimators': 1400,
    'boost_from_average': False,
    'verbose': -1,
} 

**boosting_type**
* 'gbdt' - стандартный метод для повышения градиента
* 'goss' - для более быстрого обучения, но обычно это приводит к недостаточной приспособленности
* 'dart' - для очень долгого обучения, а производительность модели зависит от случайного фактора

**objective**

Повышение градиента tweediie для чрезвычайно несбалансированных нулевых завышенных данных.

**tweedie_variance_power**

* default = 1.5
* Установите это значение ближе к 2, чтобы перейти к гамма распределению
* Установите это значение ближе к 1, чтобы перейти к распределению Пуассона
* Значение 1.1 является самым оптимальным

**metric**

Это ничего не значит для нас, так как метрика конкуренции отличается, и мы не используем здесь ранние остановки.
Таким образом, rmse служит только для общего обзора производительности модели.
Кроме того, мы используем "поддельный" набор валидации (поскольку он входит в состав тренировочного набора),
поэтому даже общий балл rmse ничего не значит.

**subsample**

Служит для борьбы с overfit
будет случайным образом выбирать часть данных без передискретизации
Был выбран CV

**subsample_freq**

частота для упаковки
default value кажется в порядке

**learning_rate**

Выбирается по CV
Smaller - дольше тренировка, но есть возможность остановиться в "локальном минимуме"
Bigger - быстрее тренировка, но есть шанс не найти "глобальный минимум" минимума

**num_leaves и min_data_in_leaf**

Принудительная модель для использования дополнительных функций
Нам это нужно, чтобы уменьшить влияние "рекурсивных" ошибок
Кроме того, это приводит к переоснащению, поэтому мы используем маленький 'max_bin': 100

**l1, l2 регуляризации**

l2 может работать с большими num_leaves
                    
**n_estimators**

CV показывает, что для каждого состояния/хранилища должны быть разные значения.
Текущее значение было выбрано для общего назначения.
Поскольку мы не используем никаких ранних остановок, осторожно, чтобы не перегружать Public LB.

**feature_fraction**

LightGBM будет случайным образом выбирать часть объектов на каждой итерации (tree).
У нас есть много функций, и многие из них являются "дубликатами", а многие просто "шумят" хорошие значения здесь - 0.5-0.7 (by CV)

**boost_from_average**

Существует некоторая "проблема" в коде boost_from_average для пользовательской потери
'True' делает обучение более быстрым, но тщательно используйте его

**Переменные**

In [ ]:
VER = 1                          # Версия нашей модели
N_CORES = psutil.cpu_count()     # Количество доступных ядер процессора

# Мы хотим, чтобы всё было как можно более детерминированным
SEED = 42
seed_everything(SEED)
lgb_params['seed'] = SEED

# Лимиты и константы
TARGET      = 'sales'            # Наша цель
START_TRAIN = 0                  # Мы можем пропустить несколько строк (Nans/faster training)
END_TRAIN   = 1913               # Последний день для тренировки
P_HORIZON   = 28                 # Количество дней, на которые создается предсказание
USE_AUX     = True               # Использование предварительного обучения модели

# Удаляемые характеристики
remove_features = [
    'id',
    'state_id',
    'store_id',
    'date',
    'wm_yr_wk',
    'd',
    TARGET
]

# Средние характеристики
mean_features = [
    'enc_cat_id_mean',
    'enc_cat_id_std',
    'enc_dept_id_mean',
    'enc_dept_id_std',
    'enc_item_id_mean',
    'enc_item_id_std'
] 

# Пути
ORIGINAL = '../input/m5-forecasting-accuracy/'
BASE     = '../input/m5-simple-fe/grid_part_1.pkl'
PRICE    = '../input/m5-simple-fe/grid_part_2.pkl'
CALENDAR = '../input/m5-simple-fe/grid_part_3.pkl'
LAGS     = '../input/m5-lags-features/lags_df_28.pkl'
MEAN_ENC = '../input/m5-custom-features/mean_encoding_df.pkl'

# AUX (предварительное обучение) Путь до модели
AUX_MODELS = '../input/m5-aux-models/'

# Идентификаторы магазинов
STORES_IDS = pd.read_csv(ORIGINAL + 'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())

# SPLIT для создания лагов
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY + N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i, j])

Aux Models

In [ ]:
# Если вы не хотите ждать часами, чтобы получить результат, 
# вы можете обучить каждое хранилище в отдельном ядре, а затем просто присоединиться к результату.

# Если мы хотим использовать предварительно обученные модели, мы можем пропустить обучение
if USE_AUX:
    lgb_params['n_estimators'] = 2

Модель обучения

In [ ]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Получить grid для текущего магазина
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Маска для обучения (все данные меньше 1913 года)
    # Валидация (последние 28 дней - не настоящий набор валидаций)
    # Тест (все данные больше 1913 дня, с некоторым пробелом для рекурсивных функций)
    train_mask = grid_df['d'] <= END_TRAIN
    valid_mask = train_mask & (grid_df['d'] > (END_TRAIN - P_HORIZON))
    preds_mask = grid_df['d'] > (END_TRAIN - 100)
    
    # Примените маски и сохраните набор данных lgb в качестве bin, чтобы уменьшить всплески памяти во время преобразования dtype
    # Чтобы избежать каких-либо конверсий, вы всегда должны использовать np.float32
    # или сохранить в bin перед началом обучения
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], label=grid_df[valid_mask][TARGET])
    
    # Сохранение части набора данных для последующих прогнозов
    # Удаление объектов, которые мы должны вычислять рекурсивно
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Запуск seeder снова, чтобы сделать обучение 100% детерминированным
    seed_everything(SEED)
    estimator = lgb.train(
        lgb_params,
        train_data,
        valid_sets = [valid_data],
        verbose_eval = 100
    )
    
    # Сохранение модели
    # estimator = lgb.Booster(model_file='model.txt')
    # можно прогнозировать только с лучшей итерацией (или с сохранением итерации)
    # pickle.dump дает нам больше гибкости
    # например estimator.predict(TEST, num_iteration=100)
    # num_iteration - количество итераций, которые вы хотите предсказать 
    # NULL или <= 0 означает использование наилучшей итерации
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Удаление временных файлов и объектов, чтобы освободить немного места на жестком диске и оперативной памяти
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # Сохранить функции моделей для предсказаний
    MODEL_FEATURES = features_columns

**Прогнозирование**

In [ ]:
# Создание фиктивного фрейма данных для хранения прогнозов
all_preds = pd.DataFrame()

# Соединие тестового набора данных с небольшой частью обучающих данных, чтобы создать рекурсивные объекты.
base_test = get_base_test()

# Таймер для измерения времени предсказаний
main_time = time.time()

# Цикл предсказания для каждого дня.
# Поскольку скользящие лаги являются наиболее трудоёмкими, мы рассчитываем его на целый день.
for PREDICT_DAY in range(1, 29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Сделайте временную сетку для расчета задержек качения
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        # Считываем все наши модели и делаем прогнозы
        model_path = 'lgb_model_' + store_id + '_v' + str(VER) + '.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES],num_iteration=1400)
    
    # Создаем столбцы и добавляем в DataFrame all_preds
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

**Экспорт**

In [ ]:
# Чтение sample_submission и слияние наших прогнозов.
# Поскольку у нас есть прогнозы только для данных "_validation", нам нужно сделать fillna() для элементов "_evaluation".
submission = pd.read_csv(ORIGINAL + 'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v' + str(VER) + '.csv', index=False)

|num_iteration|LB score|
|---|---|
|1400|0.47506|
|1300|0.47450|
|1200|0.47505|
|1100|0.47509|